## SimHash

SimHash是Google在2007年发表的论文《Detecting Near-Duplicates for Web Crawling 》中提到的一种指纹生成算法或者叫指纹提取算法，被Google广泛应用在亿级的网页去重的Job中，作为locality sensitive hash（局部敏感哈希）的一种，其主要思想是降维

<img src="../../ml/data/img/simhash.png" style="zoom:50%"/>

解释下上图：
 - （1）准备一篇文本
 - （2）过滤清洗，提取n个特征关键词，这步一般用分词的方法实现，关于分词，比较常用的有IK，mmseg4j，ansj
 - （3）特征加权，这一步如果有自己针对某个行业的定义的语料库时候可以使用，没有的话，就用分词后的词频即可
 - （4）对关键词进行hash降维01组成的签名（上述是6位）
 - （5）然后向量加权，对于每一个6位的签名的每一位，如果是1，hash和权重正相乘，如果为0，则hash和权重负相乘，至此就能得到每个特征值的向量。
 - （6）合并所有的特征向量相加，得到一个最终的向量，然后降维，对于最终的向量的每一位如果大于0则为1，否则为0，这样就能得到最终的simhash的指纹签名


<img src="../../ml/data/img/simhash_example.png" style="zoom:50%"/>


### 汉明距离
两个码字的对应比特取值不同的比特数称为这两个码字的海明距离。在一个有效编码集中,任意两个码字的海明距离的最小值称为该编码集的海明距离。举例如下：10101和00110从第一位开始依次有第一位、第四、第五位不同，则海明距离为3。

``` python
# simhash1 异或 simhash2
x = simhash1 ^ simhash2
ans = 0
while x:
    ans += 1
    x &= x - 1
# x中1的个数就是汉明距离
```
n位的码字可以用n维空间的超立方体的一个顶点来表示。两个码字之间的海明距离就是超立方体两个顶点之间的一条边，而且是这两个顶点之间的最短距离。

### 索引
针对海量数据的去重效率，我们可以将64位指纹，切分为4份16位的数据块，根据抽屉原理在海明距离为3的情况，如果两个文档相似，那么它必有一个块的数据是相等的


<img src="../../ml/data/img/simhash_index.png" style="zoom:50%"/>

<img src="../../ml/data/img/simhash_build_index.png" style="zoom:50%"/>

假设样本库，有2^34条数据（171亿数据），假设数据均匀分布，则每个16位（16个01数字随机组成的组合为2^16个）
- 倒排返回的最大数量为2^34/2^16=2^(34-16)=262144个候选结果
- 4个16位截断索引，总的结果为：4*262144=1048576，约为100多万

通过这样一来的降维处理，原来需要比较171亿次，现在只需要比较100万次即可得到结果，这样以来大大提升了计算效率。

 